In [29]:
import cv2
import IPython
import numpy as np
from typing import Tuple, Union
import math
from PIL import Image
import pytesseract
import traceback
import re
import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
import torch.nn.functional as F
import matplotlib.pyplot as plt

import torchvision.transforms as transforms

from deskew import determine_skew

import time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import easyocr
reader = easyocr.Reader(['en', 'de'])

from ultralytics import YOLO
yolo_model = YOLO('yolov8m.pt')
# plate_model = YOLO('best_plate_model.pt')
plate_model = YOLO('best_3.pt')
# plate_model = YOLO('vehicle_license_best.pt')
characters_model = YOLO('plate_char/yolov8x/weights/best.pt')
other_characters_model = YOLO('charenyeni/yolov8x/weights/best.pt')

detected_plates = []

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

target_classes = ["truck", "bus", "car", "train", "bicycle"]

characters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ" # '0123456789abcdefghijklmnopqrstuvwxyz' # "0123456789ÄÜÖABCDEFGHIJKLMNOPQRSTUVWXYZ"

# STN = STNet()
# STN.to(device)
# STN.load_state_dict(torch.load('LPRNet/weights/Final_STN_model.pth', map_location=lambda storage, loc: storage))
# STN.eval()

times = []

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)

#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)

#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def compute_skew(src_img):
    if len(src_img.shape) == 3:
        h, w, _ = src_img.shape
    elif len(src_img.shape) == 2:
        h, w = src_img.shape
    else:
        print('unsupported image type')
    img = cv2.medianBlur(src_img, 3)
    edges = cv2.Canny(img,  threshold1 = 30,  threshold2 = 100, apertureSize = 3, L2gradient = True)
    lines = cv2.HoughLinesP(edges, 1, math.pi/180, 30, minLineLength=w / 4.0, maxLineGap=h/4.0)
    angle = 0.0
    nlines = lines.size
    #print(nlines)
    cnt = 0
    for x1, y1, x2, y2 in lines[0]:
        ang = np.arctan2(y2 - y1, x2 - x1)
        #print(ang)
        if math.fabs(ang) <= 30: # excluding extreme rotations
            angle += ang
            cnt += 1
    if cnt == 0:
        return 0.0
    return (angle / cnt)*180/math.pi

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def get_deskew_angle(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle<-45: angle=-angle
    return angle

def deskew_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

# NN Layer License Plate Recognition "OCR"
def process_license(processed_frame, bounding_box, name, confidence, parentName, parentConfidence):
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(255,0,255),2)
    cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1-15),0,0.9,(255,0,255),2)

    # Rotation
    # license_frame = deskew(processed_frame[y1:y2,x1:x2])
    license_frame = processed_frame[y1:y2,x1:x2]

    plt.imshow(license_frame)
    plt.show()

    plate_parts=[]
    plate_text=""

    start_time = time.time()
    results = characters_model(license_frame, agnostic_nms=True, verbose=False, device=device)[0]
    total_time = time.time() - start_time
    # cv2.putText(processed_frame, '{:.3f}sec'.format(total_time),(x1+100,y1+200),0,0.7,(0,0,255),2)
    
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name2 = result.names[cls]
            confidence2 = float(result.boxes.conf[i].item())
            bounding_box2 = result.boxes.xyxy[i].cpu().numpy()
            a1, b1, a2, b2 = [int(x) for x in bounding_box2]
            # Ensure a1 < a2 and b1 < b2
            a1, a2 = min(a1, a2), max(a1, a2)
            b1, b2 = min(b1, b2), max(b1, b2)
            if confidence2>0.3:
                # cv2.rectangle(processed_frame,(x1+a1,y1+b1),(x1+a2,y1+b2),(0,0,255),1)
                # cv2.putText(processed_frame, '{}'.format(name2),(x1+a1,y1+b1+75),0,0.9,(0,0,255),2)
                # cv2.putText(processed_frame, '{:.2f}%'.format(confidence2*100),(x1+a1,y1+b1+100+25*i),0,0.7,(0,0,255),2)
                plate_parts.append([a1,b1,name2])

    sorted_list = sorted(plate_parts,key=lambda l:l[0])
    for part in range(len(sorted_list)):
        if sorted_list[part][2]!="undefined":
            plate_text+=sorted_list[part][2]
    print("yolov8x", plate_text, total_time)
    times.append(["yolov8x", plate_text,total_time])

    plate_parts=[]
    plate_text=""
    
    start_time = time.time()
    results = other_characters_model(license_frame, agnostic_nms=True, verbose=False, device=device)[0]
    total_time = time.time() - start_time
    # cv2.putText(processed_frame, '{:.3f}sec'.format(total_time),(x1+100,y1+200),0,0.7,(0,0,255),2)
    
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name2 = result.names[cls]
            confidence2 = float(result.boxes.conf[i].item())
            bounding_box2 = result.boxes.xyxy[i].cpu().numpy()
            a1, b1, a2, b2 = [int(x) for x in bounding_box2]
            # Ensure a1 < a2 and b1 < b2
            a1, a2 = min(a1, a2), max(a1, a2)
            b1, b2 = min(b1, b2), max(b1, b2)
            if confidence2>0.3:
                # cv2.rectangle(processed_frame,(x1+a1,y1+b1),(x1+a2,y1+b2),(0,0,255),1)
                # cv2.putText(processed_frame, '{}'.format(name2),(x1+a1,y1+b1+75),0,0.9,(0,0,255),2)
                # cv2.putText(processed_frame, '{:.2f}%'.format(confidence2*100),(x1+a1,y1+b1+100+25*i),0,0.7,(0,0,255),2)
                plate_parts.append([a1,b1,name2])

    sorted_list = sorted(plate_parts,key=lambda l:l[0])
    for part in range(len(sorted_list)):
        if sorted_list[part][2]!="undefined":
            plate_text+=sorted_list[part][2]
    print("yolov8x_charen", plate_text, total_time)
    times.append(["yolov8x_charen", plate_text,total_time])

    grayscaled = get_grayscale(license_frame) # get_grayscale(np.array(license_frame))
    # angle = determine_skew(grayscaled)
    threshholded = thresholding(grayscaled) # deskew_image(thresholding(grayscaled), angle)

    # newdata=pytesseract.image_to_osd(threshholded)
    # angle=float(re.search('(?<=Rotate: )\d+', newdata).group(0))
    # print('osd angle:',angle)

    # angle = compute_skew(thresholding(grayscaled))
    # threshholded = rotate_image(thresholding(grayscaled), angle)
    # dilated = dilate(threshholded)
    # plt.imshow(deskew(threshholded))

    # print(angle)
    plt.imshow(threshholded)
    plt.show()

    max_conf = 0.0
    best_text = ""

    start_time = time.time()
    results = pytesseract.image_to_data(threshholded, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    end_time = time.time() - start_time
    times.append(end_time)
    # NN Layer License Plate Recognition "Dummy OCR"

    parsedLines = results.split('\n')
    best_depth = len(parsedLines)-2
    for line in parsedLines:
        params = line.split()
        if len(params)==12 and float(params[10].replace('conf','0.0'))>=max_conf:
            max_conf = float(params[10].replace('conf','0.0'))
            best_text = params[11] if params[11]!="text" else "-"
            if max_conf>=0.0 and len(best_text)>0:
                detected_plates.append([parentConfidence*100, parentName.upper(), confidence*100, name.upper(), max_conf, best_text, "pytesseract"])
    # cv2.putText(processed_frame, '{} {:.2f}%'.format("[tesse]: "+best_text, max_conf),(x1+10,y2+25),0,0.9,(0,0,255),3)
    print("pytesseract", best_text, end_time)
    times.append(["pytesseract", best_text,end_time])

    max_conf = 0.0
    best_text = ""

    start_time = time.time()
    result = reader.readtext(threshholded)
    end_time = time.time() - start_time
    text = ""
    conf = 0.0
    for res in result:
        if res[2]>conf:
            conf=res[2]
            text=res[1]
            if res[2]*100>max_conf:
                max_conf = res[2]*100
                best_text = res[1]
                if max_conf>=0.0 and len(best_text)>0:
                    detected_plates.append([parentConfidence*100, parentName.upper(), confidence*100, name.upper(), max_conf, best_text, "easyocr"])
    # cv2.putText(processed_frame, '{} {:.2f}%'.format("[easy]: "+str(text), conf*100),(x1+10,y2+57),0,0.9,(200,200,200),3)
    print("easyocr", text, end_time)
    times.append(["easyocr", text,end_time])

# NN Layer Vehicle Found -> License Detection
def process_vehicle(processed_frame, bounding_box, name, confidence):

    # Yolov8n Bounding Boxes
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(0,255,255),2)

    # cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1+25),0,0.8,(0,255,255),2)
    vehicle_frame = processed_frame[y1:y2,x1:x2]
    results = plate_model(vehicle_frame, agnostic_nms=True, verbose=False)[0]
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name2 = result.names[cls]
            confidence2 = float(result.boxes.conf[i].item())
            bounding_box2 = result.boxes.xyxy[i].cpu().numpy()
            a1, b1, a2, b2 = [int(x) for x in bounding_box2]
            # Ensure a1 < a2 and b1 < b2
            a1, a2 = min(a1, a2), max(a1, a2)
            b1, b2 = min(b1, b2), max(b1, b2)
            # if name2=="License_Plate": process_license(processed_frame, [x1+a1, y1+b1, x1+a2, y1+b2], name2, confidence2, name, confidence)
            process_license(processed_frame, [x1+a1, y1+b1, x1+a2, y1+b2], name2, confidence2, name, confidence)

# NN Layer Other Objects Detected [Not target classes]
def process_detection(processed_frame, bounding_box, name, confidence):
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(255,255,0),2)
    cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1+25),0,0.8,(255,255,0),2)

# NN Layer Yolov8 Detection
def yolov8n_objects(frame):
    processed_frame = frame
    start_time = time.time()
    results = yolo_model(processed_frame, agnostic_nms=True, verbose=False)[0]
    end_time = time.time()
    processing_time = end_time - start_time
    # print(f"Processing time yolov8: {processing_time:.3f} seconds")  # Output processing time
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name = result.names[cls]
            confidence = float(result.boxes.conf[i].item())
            bounding_box = result.boxes.xyxy[i].cpu().numpy()
            x1, y1, x2, y2 = [int(x) for x in bounding_box]
            # Ensure x1 < x2 and y1 < y2
            x1, x2 = min(x1, x2), max(x1, x2)
            y1, y2 = min(y1, y2), max(y1, y2)
            if name in target_classes: process_vehicle(processed_frame, bounding_box, name, confidence)
            else: process_detection(processed_frame, bounding_box, name, confidence)
    return processed_frame

def process_frame(frame):
    processed_frame = yolov8n_objects(frame)
    # processed_frame = cv2.flip(processed_frame, 1)
    return processed_frame

def handleFrame(frame):
    processed_frame = process_frame(frame)
    _, processed_frame = cv2.imencode('.jpeg', processed_frame)
    display_handle.update(IPython.display.Image(data=processed_frame.tobytes()))
    IPython.display.clear_output(wait=True)

def handleRelease():
    cam.release()
    print("Source released.")
    correct_plates = []
    better_plates = []
    best_plate = []
    most_repeated_plates = []
    best_repeated_plates = []
    highest_score = 0.0
    if len(detected_plates)>0:
        repetitions = {}
        count, item = 0, ''
        for plate in detected_plates:
            if plate[0]>50.0 and plate[2]>50.0 and plate[4]>50.0: correct_plates.append(plate)
            if plate[0]>70.0 and plate[2]>70.0 and plate[4]>70.0: better_plates.append(plate)
            if plate[0] + plate[2] + plate[4] > highest_score:
                highest_score = plate[0] + plate[2] + plate[4]
                best_plate = plate
            repetitions[plate[5]] = repetitions.get(plate[5], 0) + 1
            if repetitions[plate[5]]>count: count, item = repetitions[plate[5]], plate[5]
        if len(repetitions.keys())>0:
            for itm in repetitions.keys():
                if repetitions[itm]==count:
                    repeated_detected = [plate for plate in detected_plates if plate[5]==itm]
                    best_percentages = []
                    for plate in repeated_detected:
                        if len(best_percentages)==0: best_percentages=plate
                        elif (plate[0] + plate[2] + plate[4])/3.0>(best_percentages[0] + best_percentages[2] + best_percentages[4])/3.0: best_percentages=plate
                    most_repeated_plates.append(best_percentages)
        if len(most_repeated_plates)>0:
            for plate in most_repeated_plates:
                best_repeated = []
                if len(best_repeated)==0: best_repeated=plate
                elif (plate[0] + plate[2] + plate[4])/3.0>(best_repeated[0] + best_repeated[2] + best_repeated[4])/3.0: best_repeated=plate
            best_repeated_plates.append(best_repeated)
    print("\nDetected Plates: \n", np.matrix(detected_plates))
    print("\nCorrectly Detected Plates: \n", np.matrix(correct_plates))
    print("\nBetter Detected Plates: \n", np.matrix(better_plates))
    print("\nBest Detected Plate: \n", np.matrix(best_plate))
    print("\nMost Repeated Plates: \n", np.matrix(most_repeated_plates))
    print("\nBest Repeated Plate: \n", np.matrix(best_repeated_plates))

    print("times",times)

inputType="video"
# cam = cv2.VideoCapture(0)
source = "data/video/sample.MP4" if inputType=="video" else 0
cam = cv2.VideoCapture(source)
display_handle=display(None, display_id=True)
every = 60

start = 0 if source!=0 else -1
end = int(cam.get(cv2.CAP_PROP_FRAME_COUNT)) if source!=0 else -1
if source!=0: cam.set(1, start)

frameCount=start
while_safety=0
saved_count=0

if not cam.isOpened(): print("Error: Could not open source.")
else:
    try:
        while True if end<0 else frameCount<end:
            _, frame = cam.read()
            if frame is None:
                if source!=0:
                    if while_safety > 2000: break
                    while_safety += 1
                    continue
                else:
                    print("Error: Could not capture frame.")
                    cam.release()
                    break

            if every>0:
                if (frameCount+1)%math.floor(every) == 0:
                    while_safety = 0
                    handleFrame(frame)
            else:
                while_safety=0
                handleFrame(frame)
            frameCount += 1
        handleRelease()
    except KeyboardInterrupt:
        handleRelease()
    except:
        print("Unknown error.")
        try:
            cam.release()
            raise TypeError("Error: Source could not be released.")
        except:
            pass
        traceback.print_exc()

Source released.

Detected Plates: 
 [['65.81149101257324' 'BUS' '84.66718792915344' ... '0.0' '-' 'pytesseract']
 ['65.81149101257324' 'BUS' '84.66718792915344' ... '0.0' 'WAFSHW379' 'pytesseract']
 ['65.81149101257324' 'BUS' '84.66718792915344' ... '28.131838772282723' 'hh379' 'easyocr']
 ...
 ['84.02048945426941' 'TRUCK' '70.30436396598816' ... '0.0' '-' 'pytesseract']
 ['84.02048945426941' 'TRUCK' '70.30436396598816' ... '24.132578362115062' 'JyEcg' 'easyocr']
 ['84.02048945426941' 'TRUCK' '39.7539347410202' ... '0.0' '-' 'pytesseract']]

Correctly Detected Plates: 
 [['59.67785120010376' 'BUS' '82.01947212219238' 'LICENSE-PLATE' '60.83841044778365' 'HAFshk 379' 'easyocr']
 ['76.15963220596313' 'BUS' '83.02473425865173' 'LICENSE-PLATE' '62.22941007807742' 'KAFshH 379' 'easyocr']
 ['90.1332139968872' 'TRUCK' '80.19434213638306' 'LICENSE-PLATE' '64.08528222719607' 'IGH 3353A' 'easyocr']
 ['95.09167075157166' 'TRUCK' '81.67493343353271' 'LICENSE-PLATE' '60.094541549777524' 'S0' 'easyo

In [23]:
# Neuronal Network Layered Stream with the better fps
import cv2
import IPython
import numpy as np
from typing import Tuple, Union
import math
from PIL import Image
# import pytesseract
import traceback
import time

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from ultralytics import YOLO
model = YOLO('yolov8x.pt')
plate_model = YOLO('vehicle_license_best.pt')
characters_model = YOLO('plate_char/yolov8x/weights/best.pt')

detected_plates = []

# pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

target_classes = ["truck", "bus", "car", "train", "bicycle"]

# NN Layer License Plate Recognition "OCR"
def process_license(processed_frame, bounding_box, name, confidence, parentName, parentConfidence, start_time, step_time):
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(255,0,255),2)
    cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1-15),0,0.7,(255,0,255))

    license_frame = processed_frame[y1:y2,x1:x2]

    plate_parts=[]
    plate_text=""

    results = characters_model(license_frame, agnostic_nms=True, verbose=False, device=device)[0]
    # total_time = time.time() - start_time
    # cv2.putText(processed_frame, '{:.3f}sec'.format(total_time),(x1+100,y1+200),0,0.7,(0,0,255),2)
    
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name2 = result.names[cls]
            confidence2 = float(result.boxes.conf[i].item())
            bounding_box2 = result.boxes.xyxy[i].cpu().numpy()
            a1, b1, a2, b2 = [int(x) for x in bounding_box2]
            # Ensure a1 < a2 and b1 < b2
            a1, a2 = min(a1, a2), max(a1, a2)
            b1, b2 = min(b1, b2), max(b1, b2)
            if confidence2>0.3:
                cv2.rectangle(processed_frame,(x1+a1,y1+b1),(x1+a2,y1+b2),(0,0,255),1)
                cv2.putText(processed_frame, '{}'.format(name2),(x1+a1,y1+b1+75),0,0.9,(0,0,255),2)
                cv2.putText(processed_frame, '{:.2f}%'.format(confidence2*100),(x1+a1,y1+b1+100+25*i),0,0.7,(0,0,255),2)
                plate_parts.append([a1,b1,name2])

    sorted_list = sorted(plate_parts,key=lambda l:l[0])
    for part in range(len(sorted_list)):
        if sorted_list[part][2]!="undefined":
            plate_text+=sorted_list[part][2]
    detected_plates.append(plate_text)

# NN Layer Vehicle Found -> License Detection
def process_vehicle(processed_frame, bounding_box, name, confidence, start_time):
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(0,255,255),2)
    cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1-15),0,0.7,(0,255,255))
    vehicle_frame = processed_frame[y1:y2,x1:x2]
    step_time = time.time()
    results = plate_model(vehicle_frame, agnostic_nms=True, verbose=False, device=device)[0]
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name2 = result.names[cls]
            confidence2 = float(result.boxes.conf[i].item())

            if confidence2 > 0.5:
                bounding_box2 = result.boxes.xyxy[i].cpu().numpy()
                a1, b1, a2, b2 = [int(x) for x in bounding_box2]
                # Ensure a1 < a2 and b1 < b2
                a1, a2 = min(a1, a2), max(a1, a2)
                b1, b2 = min(b1, b2), max(b1, b2)
                # if name2=="License_Plate":
                process_license(processed_frame, [x1+a1, y1+b1, x1+a2, y1+b2], name2, confidence2, name, confidence, start_time, step_time)

# NN Layer Other Objects Detected
def process_detection(processed_frame, bounding_box, name, confidence):
    x1, y1, x2, y2 = [int(x) for x in bounding_box]
    # cv2.rectangle(processed_frame,(x1,y1),(x2,y2),(255,255,0),2)
    # cv2.putText(processed_frame, '{} {:.2f}%'.format(name.upper(), confidence*100),(x1+10,y1-15),0,0.7,(255,255,0))

# NN Layer Yolov8 Detection
def process_frame(frame):
    processed_frame = frame
    # processed_frame = cv2.flip(processed_frame, 1)
    start_time = time.time()
    results = model(processed_frame, agnostic_nms=True, verbose=False, device=device)[0]
    for result in results:
        detection_count = result.boxes.shape[0]
        for i in range(detection_count):
            cls = int(result.boxes.cls[i].item())
            name = result.names[cls]
            confidence = float(result.boxes.conf[i].item())
            bounding_box = result.boxes.xyxy[i].cpu().numpy()
            x1, y1, x2, y2 = [int(x) for x in bounding_box]
            # Ensure x1 < x2 and y1 < y2
            x1, x2 = min(x1, x2), max(x1, x2)
            y1, y2 = min(y1, y2), max(y1, y2)
            if name in target_classes: process_vehicle(processed_frame, bounding_box, name, confidence, start_time)
            else: process_detection(processed_frame, bounding_box, name, confidence)
    return processed_frame

def handleFrame(frame):
    processed_frame = process_frame(frame)
    _, processed_frame = cv2.imencode('.jpeg', processed_frame)
    display_handle.update(IPython.display.Image(data=processed_frame.tobytes()))
    IPython.display.clear_output(wait=True)

def handleRelease():
    cam.release()
    print("Source released.")
    print()
    print(detected_plates)
    # correct_plates = []
    # better_plates = []
    # best_plate = []
    # most_repeated_plates = []
    # best_repeated_plates = []
    # highest_score = 0.0
    # if len(detected_plates)>0:
    #     repetitions = {}
    #     count, item = 0, ''
    #     for plate in detected_plates:
    #         if plate[0]>50.0 and plate[2]>50.0 and plate[4]>50.0: correct_plates.append(plate)
    #         if plate[0]>70.0 and plate[2]>70.0 and plate[4]>70.0: better_plates.append(plate)
    #         if plate[0] + plate[2] + plate[4] > highest_score:
    #             highest_score = plate[0] + plate[2] + plate[4]
    #             best_plate = plate
    #         repetitions[plate[5]] = repetitions.get(plate[5], 0) + 1
    #         if repetitions[plate[5]]>count: count, item = repetitions[plate[5]], plate[5]
    #     if len(repetitions.keys())>0:
    #         for itm in repetitions.keys():
    #             if repetitions[itm]==count:
    #                 repeated_detected = [plate for plate in detected_plates if plate[5]==itm]
    #                 best_percentages = []
    #                 for plate in repeated_detected:
    #                     if len(best_percentages)==0: best_percentages=plate
    #                     elif (plate[0] + plate[2] + plate[4])/3.0>(best_percentages[0] + best_percentages[2] + best_percentages[4])/3.0: best_percentages=plate
    #                 most_repeated_plates.append(best_percentages)
    #     if len(most_repeated_plates)>0:
    #         for plate in most_repeated_plates:
    #             best_repeated = []
    #             if len(best_repeated)==0: best_repeated=plate
    #             elif (plate[0] + plate[2] + plate[4])/3.0>(best_repeated[0] + best_repeated[2] + best_repeated[4])/3.0: best_repeated=plate
    #         best_repeated_plates.append(best_repeated)
    # print("\nDetected Plates: \n", np.matrix(detected_plates))
    # print("\nCorrectly Detected Plates: \n", np.matrix(correct_plates))
    # print("\nBetter Detected Plates: \n", np.matrix(better_plates))
    # print("\nBest Detected Plate: \n", np.matrix(best_plate))
    # print("\nMost Repeated Plates: \n", np.matrix(most_repeated_plates))
    # print("\nBest Repeated Plate: \n", np.matrix(best_repeated_plates))

inputType="video"
# cam = cv2.VideoCapture(0)
source = "data/video/vecteezy_car-and-truck-traffic-on-the-highway-in-europe-poland_7957364.MP4" if inputType=="video" else 0
cam = cv2.VideoCapture(source)
display_handle=display(None, display_id=True)
every = 30

start = 0 if source!=0 else -1
end = int(cam.get(cv2.CAP_PROP_FRAME_COUNT)) if source!=0 else -1
if source!=0: cam.set(1, start)

frameCount=start
while_safety=0
saved_count=0

if not cam.isOpened(): print("Error: Could not open source.")
else:
    try:
        while True if end<0 else frameCount<end:
            _, frame = cam.read()
            if frame is None:
                if source!=0:
                    if while_safety > 2000: break
                    while_safety += 1
                    continue
                else:
                    print("Error: Could not capture frame.")
                    cam.release()
                    break

            if every>0:
                if (frameCount+1)%math.floor(every) == 0:
                    while_safety = 0
                    handleFrame(frame)
            else:
                while_safety=0
                handleFrame(frame)
            frameCount += 1
        handleRelease()
    except KeyboardInterrupt:
        handleRelease()
    except:
        print("Unknown error.")
        try:
            cam.release()
            raise TypeError("Error: Source could not be released.")
        except:
            pass
        traceback.print_exc()

Source released.

['K0S0922E', 'BA544', 'WI460KA', '0PR8X14', '', 'K0S0922E', 'BA25466', 'AY', 'SB3UJ7', 'OPR8X14', 'SJ248V']


In [2]:
import torchvision
from torchvision.models.detection import *

 # DetectionModel("FCOS", "resnet50_fpn", fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights, "fcos_resnet", (255,0,0)),
 #    DetectionModel("RetinaNet", "resnet50_fpn", retinanet_resnet50_fpn, RetinaNet_ResNet50_FPN_Weights, "retinanet_resnet", (255,255,0)),
 #    DetectionModel("RetinaNet", "resnet50_fpn_v2", retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights, "retinanet_resnet_v2", (0,234,255)),
 #    DetectionModel("FasterRCNN", "resnet50_fpn", fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights, "faster_rcnn_resnet", (170,0,255)),
 #    DetectionModel("FasterRCNN", "resnet50_fpn_v2", fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights, "faster_rcnn_resnet_v2", (255,127,0)),
 #    DetectionModel("FasterRCNN", "mobilenet_v3_large_fpn", fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights, "faster_rcnn_mobilenet_v3", (191,255,0)),
 #    DetectionModel("FasterRCNN", "mobilenet_v3_large_320_fpn", fasterrcnn_mobilenet_v3_large_320_fpn, FasterRCNN_MobileNet_V3_Large_320_FPN_Weights, "faster_rcnn_mobilenet_v3_320", (0,149,255)),
 #    DetectionModel("MaskRCNN", "resnet50_fpn", maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights, "mask_rcnn_resnet", (255,0,170)),
 #    DetectionModel("MaskRCNN", "resnet50_fpn_v2", maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights, "mask_rcnn_resnet_v2", (255,212,0)),
 #    DetectionModel("SSD300", "vgg16", ssd300_vgg16, SSD300_VGG16_Weights, "ssd_vgg16", (106,255,0)),
 #    DetectionModel("SSDLite320", "mobilenet_v3_large", ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights, "ssd_mobilenet_v3", (0,64,255)),

model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()

# Load pre-trained Faster R-CNN
# model_faster_rcnn = fasterrcnn_resnet50_fpn(pretrained=True)
# model_faster_rcnn.eval()

# Load pre-trained Mask R-CNN
# model_mask_rcnn = maskrcnn_resnet50_fpn(pretrained=True)
# model_mask_rcnn.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(